In [1]:
import os

In [4]:
%pwd # print working directory

'd:\\Downloads\\Chest-CT-Classification\\Chest-CT-Classification-MLflow-DVC'

In [3]:
os.chdir("../") # change working directory to previous directory

In [5]:
from dataclasses import dataclass
from pathlib import Path

# in dataclass we dont mention self as it is automatically added
# frozen=True makes the class immutable
# same parameters should be used in the config.yaml file

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
                 ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
 
        create_directories([self.config.artifacts_root])  # creates data directory

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        '''Fetches data from the url'''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to {zip_download_dir}")

            fileid = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + fileid, zip_download_dir)

            logger.info(f"Downloaded data to {zip_download_dir}")

        except Exception as e:
            # logger.error(f"Error downloading data: {e}")
            raise e
        
    def extract_zip_file(self):
        """ 
        zip_file: Path to the zip file
        Extracts the zip file to the directory
        Function return None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e
    

[2024-08-25 23:42:47,671:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-08-25 23:42:47,679:INFO:common:yaml file: params.yaml loaded successfully]
[2024-08-25 23:42:47,683:INFO:common:Created directory: artifacts]
[2024-08-25 23:42:47,686:INFO:common:Created directory: artifacts/data_ingestion]
[2024-08-25 23:42:47,687:INFO:3207028589:Downloading data from https://drive.google.com/file/d/1exE9Yiwg07zKRmuquIU94T2a1K_Qgo9V/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1exE9Yiwg07zKRmuquIU94T2a1K_Qgo9V
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1exE9Yiwg07zKRmuquIU94T2a1K_Qgo9V&confirm=t&uuid=051ab8b5-1d24-42f2-8381-a0fdcba71128
To: d:\Downloads\Chest-CT-Classification\Chest-CT-Classification-MLflow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [00:14<00:00, 8.87MB/s] 

[2024-08-25 23:43:08,337:INFO:3207028589:Downloaded data to artifacts/data_ingestion/data.zip]
